In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')
import re

# Read the URL file into the pandas object
input_df = pd.read_excel('/content/drive/MyDrive/Blackcoffer/Input.xlsx')

# Create the directory for text files if it doesn't exist
content_folder = '/content/drive/MyDrive/Blackcoffer/TextTitle/'
os.makedirs(content_folder, exist_ok=True)

# Loop through each row in the dataframe
for idx, row in input_df.iterrows():
    web_link = row['URL']
    page_id = row['URL_ID']

    # Make a request to the URL
    user_agent = {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36"}
    try:
        web_response = requests.get(web_link, headers=user_agent)
    except:
        print(f"Failed to fetch response for {page_id}")
        continue

    # Create a BeautifulSoup object
    try:
        page_content = BeautifulSoup(web_response.content, 'html.parser')
    except:
        print(f"Failed to load page for {page_id}")
        continue

    # Extract title
    try:
        page_title = page_content.find('h1').get_text()
    except:
        print(f"Failed to extract title for {page_id}")
        continue

    # Extract text content
    text_content = ""
    try:
        for para in page_content.find_all('p'):
            text_content += para.get_text()
    except:
        print(f"Failed to extract text for {page_id}")
        continue

    # Save title and text to a file
    output_file = os.path.join(content_folder, f'{page_id}.txt')
    with open(output_file, 'w') as file:
        file.write(page_title + '\n' + text_content)

# Directories
text_content_dir = "/content/drive/MyDrive/Blackcoffer/TextTitle/"
stopword_path = "/content/drive/MyDrive/Blackcoffer/StopWords"
sentiment_path = "/content/drive/MyDrive/Blackcoffer/MasterDictionary"

# Load stopwords from directory and store in a set
custom_stopwords = set()
for stopword_file in os.listdir(stopword_path):
    with open(os.path.join(stopword_path, stopword_file), 'r', encoding='ISO-8859-1') as f:
        custom_stopwords.update(set(f.read().splitlines()))

# Load all text files from the directory and store them in a list (cleaned_docs)
cleaned_docs = []
for doc_file in os.listdir(text_content_dir):
    with open(os.path.join(text_content_dir, doc_file), 'r') as f:
        doc_text = f.read()
        # Tokenize the text
        tokens = word_tokenize(doc_text)
        # Remove stopwords from tokens
        filtered_words = [word for word in tokens if word.lower() not in custom_stopwords]
        # Add filtered tokens to the list
        cleaned_docs.append(filtered_words)

# Load positive and negative words from the sentiment dictionary
positive_words_set = set()
negative_words_set = set()

for dict_file in os.listdir(sentiment_path):
    if dict_file == 'positive-words.txt':
        with open(os.path.join(sentiment_path, dict_file), 'r', encoding='ISO-8859-1') as f:
            positive_words_set.update(f.read().splitlines())
    elif dict_file == 'negative-words.txt':
        with open(os.path.join(sentiment_path, dict_file), 'r', encoding='ISO-8859-1') as f:
            negative_words_set.update(f.read().splitlines())

# Analyze positive and negative words from each document
pos_words_list = []
neg_words_list = []
positive_scores = []
negative_scores = []
polarity_scores = []
subjectivity_scores = []

for doc in cleaned_docs:
    pos_words_list.append([word for word in doc if word.lower() in positive_words_set])
    neg_words_list.append([word for word in doc if word.lower() in negative_words_set])
    positive_scores.append(len(pos_words_list[-1]))
    negative_scores.append(len(neg_words_list[-1]))
    polarity_scores.append((positive_scores[-1] - negative_scores[-1]) / ((positive_scores[-1] + negative_scores[-1]) + 0.000001))
    subjectivity_scores.append((positive_scores[-1] + negative_scores[-1]) / (len(doc) + 0.000001))

# Function to calculate readability metrics
def calculate_readability_metrics(doc_file):
    with open(os.path.join(text_content_dir, doc_file), 'r') as f:
        doc_text = f.read()
        doc_text = re.sub(r'[^\w\s.]', '', doc_text)
        sentences = doc_text.split('.')
        total_sentences = len(sentences)
        words_in_doc = [word for word in doc_text.split() if word.lower() not in custom_stopwords]
        total_words = len(words_in_doc)

        # Identify complex words (with more than 2 syllables)
        complex_words = []
        for word in words_in_doc:
            syllable_count = sum(1 for letter in word if letter.lower() in 'aeiou')
            if syllable_count > 2:
                complex_words.append(word)

        # Calculate average syllables per word
        syllable_sum = 0
        syllable_word_list = []
        for word in words_in_doc:
            syllable_count = sum(1 for letter in word if letter.lower() in 'aeiou')
            if syllable_count >= 1:
                syllable_word_list.append(word)
                syllable_sum += syllable_count

        avg_sentence_length = total_words / total_sentences
        avg_syllables_per_word = syllable_sum / len(syllable_word_list)
        percent_complex_words = len(complex_words) / total_words
        fog_score = 0.4 * (avg_sentence_length + percent_complex_words)

        return avg_sentence_length, percent_complex_words, fog_score, len(complex_words), avg_syllables_per_word

# Calculate metrics for each document
avg_sentence_lengths = []
complex_word_percentages = []
fog_indexes = []
complex_word_counts = []
syllable_per_word_avg = []

for doc_file in os.listdir(text_content_dir):
    a, b, c, d, e = calculate_readability_metrics(doc_file)
    avg_sentence_lengths.append(a)
    complex_word_percentages.append(b)
    fog_indexes.append(c)
    complex_word_counts.append(d)
    syllable_per_word_avg.append(e)

# Function to clean and calculate word count and average word length
def count_cleaned_words(doc_file):
    with open(os.path.join(text_content_dir, doc_file), 'r') as f:
        doc_text = f.read()
        doc_text = re.sub(r'[^\w\s]', '', doc_text)
        words = [word for word in doc_text.split() if word.lower() not in custom_stopwords]
        total_characters = sum(len(word) for word in words)
        avg_word_length = total_characters / len(words)
    return len(words), avg_word_length

total_word_count = []
avg_word_lengths = []

for doc_file in os.listdir(text_content_dir):
    wc, awl = count_cleaned_words(doc_file)
    total_word_count.append(wc)
    avg_word_lengths.append(awl)

# Function to count personal pronouns
def find_personal_pronouns(doc_file):
    with open(os.path.join(text_content_dir, doc_file), 'r') as f:
        doc_text = f.read()
        personal_pronouns_list = ["I", "we", "my", "ours", "us"]
        pronoun_count = 0
        for pronoun in personal_pronouns_list:
            pronoun_count += len(re.findall(r"\b" + pronoun + r"\b", doc_text))
    return pronoun_count

pronoun_counts = []

for doc_file in os.listdir(text_content_dir):
    pronoun_counts.append(find_personal_pronouns(doc_file))

# Load output DataFrame
output_df = pd.read_excel('/content/drive/MyDrive/Blackcoffer/Output Data Structure.xlsx')

# These are the required variables
metrics_list = [
    positive_scores, negative_scores, polarity_scores, subjectivity_scores,
    avg_sentence_lengths, complex_word_percentages, fog_indexes,
    avg_sentence_lengths, complex_word_counts, total_word_count, syllable_per_word_avg,
    pronoun_counts, avg_word_lengths
]

# Write the values to the output dataframe
for i, metric in enumerate(metrics_list):
    if len(metric) == len(output_df):
        output_df.iloc[:, i+2] = metric
    else:
        print(f"Error: Length mismatch for metric {i+2}")

# Save the final output
output_df.to_csv('/content/drive/MyDrive/Blackcoffer/processed_output.csv')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
